In [5]:
from random import randint as rnd, choice
import tkinter as tk
import math
import time


root = tk.Tk()
root.geometry('800x600')                                          #размер окна 
root.title("Gun Game With Points: Revenge of Triangles")          #заголовок окна
canv = tk.Canvas(root, width = 800, height = 600, bg = 'white')
canv.pack()

class arrow():
    def __init__(self, x, y, angle):       #координаты центра пушки и угол её поворота
        self.x = x                         
        self.y = y
        self.Dx = 20*math.cos(angle)
        self.Dy = 20*math.sin(angle)
        self.vx = 0
        self.vy = 0
        self.lifetime = 150                 #время жизни
        self.id = canv.create_line([self.x, self.y], 
                                   [self.x + self.Dx, self.y + self.Dy], 
                                   width = 3, arrow = tk.LAST, fill = 'brown')
        
    def move(self):
        dt = 0.2                            #квант времени

        self.x += self.vx*dt
        self.y += self.vy*dt

        canv.coords(self.id,
                    self.x, self.y, 
                    self.x + self.Dx, self.y + self.Dy)

        self.lifetime -= 1

    def hit_test(self, obj):
        if (((self.x + self.Dx) - obj.x)**2 + ((self.y + self.Dy) - obj.y)**2) <= obj.r**2:
            return True
        return False
    
class shell():
    def __init__(self, x, y):
        self.x = x                         #координаты центра пушки
        self.y = y
        self.r = 5                         #радиус по умолчанию 10   
        self.vx = 0
        self.vy = 0
        self.lifetime = 150                 #время жизни
        self.id = canv.create_oval([self.x - self.r, self.y - self.r],
                                   [self.x + self.r, self.y + self.r],
                                   fill = 'gray')
        
    def move(self):
        dt = 0.2                            #квант времени

        self.x += self.vx*dt
        self.y += self.vy*dt

        canv.coords(self.id,
                    self.x - self.r, self.y - self.r,
                    self.x + self.r, self.y + self.r)

        self.lifetime -= 1

    def hit_test(self, obj):
        if ((self.x - obj.x)**2 + (self.y - obj.y)**2) <= (obj.r + self.r)**2:
            return True
        return False

class target_straight():
    def __init__(self, x, y):
        self.set = 1
        self.x = x
        self.y = y
        self.r = rnd(10, 50)
        self.vx = rnd(20, 70)
        self.vy = rnd(20, 70)
        self.color = choice(['purple', 'green', 'red', 'cyan', 'blue'])
        self.id = canv.create_oval([self.x - self.r, self.y - self.r],
                                   [self.x + self.r, self.y + self.r], fill = self.color)

    def hit(self):
        canv.delete(self.id)
        
    def move(self):
        if self.set:
            dt = 0.2                            #квант времени

            if self.x > 750:
                self.vx *= -1
                self.x = 750
            elif self.x < 50:
                self.vx *= -1
                self.x = 50
            elif self.y > 550:
                self.vy *= -1
                self.y = 550
            elif self.y < 50:
                self.vy *= -1
                self.y = 50

            self.x += self.vx*dt
            self.y += self.vy*dt

            canv.coords(self.id,
                        self.x - self.r, self.y - self.r,
                        self.x + self.r, self.y + self.r)

class target_round():
    def __init__(self):
        self.set = 0
        self.x = 0
        self.y = 0
        self.r = 10
        self.x_rot = 0
        self.y_rot = 0
        self.r_rot = 0
        self.a = 0
        self.id = canv.create_oval(0, 0, 0, 0, fill = 'goldenrod')

    def new_target(self):
        self.set = 1
        self.x_rot = rnd(50, 750)
        self.y_rot = rnd(50, 550)
        self.r_rot = rnd(20, 50)
        self.a = choice([0.1, -0.1])
        self.x = self.x_rot + self.r_rot
        self.y = self.y_rot
        
        canv.coords(self.id,
                    self.x - self.r, self.y - self.r,
                    self.x + self.r, self.y + self.r)

    def hit(self):
        canv.coords(self.id, 0, 0,
                             0, 0)
        
    def move(self):
        if self.set:

            x_tmp = (self.x - self.x_rot)*math.cos(self.a) - (self.y - self.y_rot)*math.sin(self.a) + self.x_rot
            y_tmp = (self.x - self.x_rot)*math.sin(self.a) + (self.y - self.y_rot)*math.cos(self.a) + self.y_rot
            self.x = int(x_tmp)
            self.y = int(y_tmp)  

            canv.coords(self.id,
                        self.x - self.r, self.y - self.r,
                        self.x + self.r, self.y + self.r)
            
class hive():
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.count = 150
        self.flies = []
        self.id = canv.create_polygon([self.x - 60, self.y + 30], 
                                      [self.x + 60, self.y + 30], 
                                      [self.x, self.y - 60], 
                                      fill = 'khaki3')
    def born(self):
        if self.count == 0:
            fly = target_straight(self.x, self.y)
            self.flies += [fly]
            self.count = 150
            
class gun():
    def __init__(self, x, y):
        self.points = 0
        self.score = canv.create_text(x, 20, text = '', font = '28')
        self.bullets = []
        self.fire_power = 20
        self.fire_on = 0
        self.angle = -math.pi/2
        self.x = x
        self.y = y
        self.body = canv.create_rectangle([self.x - 10, self.y - 10], 
                                          [self.x + 10, self.y + 10], fill = 'black')
        self.id = canv.create_line([self.x, self.y], 
                                   [self.x, self.y - 20], width = 7, fill = 'black')
        
    def fire_start_crossbow(self, event):
        self.fire_on = 1

    def fire_end_crossbow(self, event):
        new_bullet = arrow(self.x, self.y, self.angle)
        self.bullets += [new_bullet]
        new_bullet.vx = self.fire_power * math.cos(self.angle)
        new_bullet.vy = self.fire_power * math.sin(self.angle)
        self.fire_on = 0
        self.fire_power = 20
        
    def fire_start_shotgun(self, event):
        self.fire_on = 1

    def fire_end_shotgun(self, event):
        mid_bullet = shell(self.x, self.y)
        self.bullets += [mid_bullet]
        mid_bullet.vx = self.fire_power * math.cos(self.angle)
        mid_bullet.vy = self.fire_power * math.sin(self.angle)
        
        top_bullet = shell(self.x, self.y)
        self.bullets += [top_bullet]
        top_bullet.vx = self.fire_power * math.cos(self.angle + (math.pi/10))
        top_bullet.vy = self.fire_power * math.sin(self.angle + (math.pi/10))
        
        bot_bullet = shell(self.x, self.y)
        self.bullets += [bot_bullet]
        bot_bullet.vx = self.fire_power * math.cos(self.angle - (math.pi/10))
        bot_bullet.vy = self.fire_power * math.sin(self.angle - (math.pi/10))
        
        self.fire_on = 0
        self.fire_power = 20
        
    def targetting_up(self, event):
        self.angle -= 0.1        
        canv.coords(self.id, self.x, self.y,
                             self.x + self.fire_power * math.cos(self.angle),
                             self.y + self.fire_power * math.sin(self.angle))

    def targetting_down(self, event):
        self.angle += 0.1
        canv.coords(self.id, self.x, self.y,
                             self.x + self.fire_power * math.cos(self.angle),
                             self.y + self.fire_power * math.sin(self.angle))

    def power_up(self):
        if self.fire_on: 
            if (self.fire_power < 100):
                self.fire_power += 1
            canv.itemconfig(self.id, fill = 'orange')
            canv.coords(self.id, self.x, self.y,
                                 self.x + self.fire_power * math.cos(self.angle),
                                 self.y + self.fire_power * math.sin(self.angle))
        else:
            canv.itemconfig(self.id, fill = 'black')
            canv.coords(self.id, self.x, self.y,
                                 self.x + self.fire_power * math.cos(self.angle),
                                 self.y + self.fire_power * math.sin(self.angle))
            
    def move_up(self, event):
        self.y -= 2
        canv.coords(self.body, self.x - 10, self.y - 10,
                               self.x + 10, self.y + 10)
        canv.coords(self.id, self.x, self.y,
                             self.x + self.fire_power * math.cos(self.angle),
                             self.y + self.fire_power * math.sin(self.angle))
            
    def move_down(self, event):
        self.y += 2
        canv.coords(self.body, self.x - 10, self.y - 10,
                               self.x + 10, self.y + 10)
        canv.coords(self.id, self.x, self.y,
                             self.x + self.fire_power * math.cos(self.angle),
                             self.y + self.fire_power * math.sin(self.angle))
        
            
g1 = gun(20, 300)
g2 = gun(780, 300)

h = hive(400, 300)

t1 = target_round()
t2 = target_round()
t3 = target_round()

root.bind('<w>', g1.move_up)
root.bind('<s>', g1.move_down)
root.bind('<a>', g1.targetting_up)
root.bind('<d>', g1.targetting_down)
    
root.bind('<Up>', g2.move_up)
root.bind('<Down>', g2.move_down)
root.bind('<Left>', g2.targetting_up)
root.bind('<Right>', g2.targetting_down)
    
root.bind('<j>', g1.fire_start_crossbow)
root.bind('<KeyRelease-j>', g1.fire_end_crossbow)
root.bind('<k>', g1.fire_start_shotgun)
root.bind('<KeyRelease-k>', g1.fire_end_shotgun)
        
root.bind('<z>', g2.fire_start_crossbow)
root.bind('<KeyRelease-z>', g2.fire_end_crossbow)
root.bind('<x>', g2.fire_start_shotgun)
root.bind('<KeyRelease-x>', g2.fire_end_shotgun)

def new_game(event = ''):
    global g1, g2, h, t1, t2, t3
        
    t1.new_target()
    t2.new_target()
    t3.new_target()

    while t1.set or t2.set or t3.set:
        t1.move()
        t2.move()
        t3.move()
        
        h.count -= 1
        h.born()
        
        for f in h.flies:
            f.move()
        
        for b in g1.bullets:
            b.move()
            if b.hit_test(t1) and t1.set:
                t1.set = 0
                t1.hit()
                g1.points += 2
            if b.hit_test(t2) and t2.set:
                t2.set = 0
                t2.hit()
                g1.points += 2
            if b.hit_test(t3) and t3.set:
                t3.set = 0
                t3.hit()
                g1.points += 2
            for f in h.flies:
                if b.hit_test(f) and f.set:
                    f.set = 0
                    f.hit()
                    h.flies.remove(f)
                    g1.points += 1
            if b.lifetime == 0:
                g1.bullets.remove(b)
                canv.delete(b.id)

        for b in g2.bullets:
            b.move()
            if b.hit_test(t1) and t1.set:
                t1.set = 0
                t1.hit()
                g2.points += 2
            if b.hit_test(t2) and t2.set:
                t2.set = 0
                t2.hit()
                g2.points += 2
            if b.hit_test(t3) and t3.set:
                t3.set = 0
                t3.hit()
                g2.points += 2
            for f in h.flies:
                if b.hit_test(f) and f.set:
                    f.set = 0
                    f.hit()
                    h.flies.remove(f)
                    g2.points += 1
            if b.lifetime == 0:
                g2.bullets.remove(b)
                canv.delete(b.id)
                
        canv.itemconfig(g1.score, text = str(g1.points))
        canv.itemconfig(g2.score, text = str(g2.points))
        time.sleep(0.03)
        g1.power_up()
        g2.power_up()
        canv.update()
    root.after(750, new_game)

new_game()

root.mainloop()

TclError: invalid command name ".!canvas"